In [1]:
# basic module
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import shutil
import glob
import time
import pickle

plt.style.use('seaborn-dark')
%config InlineBackend.figure_format = 'retina'
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20

plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.color'] = 'r'
plt.rcParams['axes.grid'] = False


# image module
import cv2
from pydicom import dcmread
import pylibjpeg
from PIL import Image
import pydicom as dcm
from pydicom import dcmread
from pydicom.data import get_testdata_file



# dataset
df = pd.read_csv('dataset.csv',index_col=0)

In [2]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [4]:
model = VGG16()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [6]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

x = Flatten()(model.output)
predictions = Dense(7, activation='softmax')(x) # output_shape 
custom_model = Model(model.input, predictions)

In [7]:
custom_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
dcm_list = df['orginal_dcm_file']
mask_path_list = list(df['body_part_file'])
img_path_list = []

for i in dcm_list:
    png_img = i.replace('.dcm','.png')
    img_path_list.append(png_img)

In [9]:
import numpy as np

def rv_3ch(og_img_path, mask_path, size):
    
    #module import
    import cv2
    
    # im_read
    og_img = cv2.imread(og_img_path)
    mask = cv2.imread(mask_path)
    
    # clahe
    clahe = cv2.createCLAHE(5.0,(8,8))
    
    ## og_img_convert
    cvt_img = cv2.resize(og_img, size)
    cvt_img = cv2.cvtColor(cvt_img,cv2.COLOR_BGR2GRAY)
    cvt_img = clahe.apply(cvt_img)  # 1 channel
    cvt_img = np.repeat(cvt_img[:,:,np.newaxis],3,-1)

    ## mask
    cvt_mask = cv2.resize(mask, size)
    cvt_mask = cv2.cvtColor(cvt_mask, cv2.COLOR_BGR2GRAY)

    ## remove
    remove = cv2.bitwise_and(cvt_img, cvt_img, mask = cvt_mask)
    
    return remove
#     print(remove.shape)
#     plt.imshow(remove)

In [10]:
og_img = cv2.imread(img_path_list[0])
og_img.shape

(1131, 760, 3)

In [11]:
rv_3ch(img_path_list[0] , mask_path_list[0] ,(224,224)).shape

(224, 224, 3)

In [12]:
from tqdm import tqdm
X = np.ndarray((4000,224,224,3))

for i in tqdm(range(len(X))):
    X[i][:][:][:] = rv_3ch(img_path_list[i], mask_path_list[i], (224,224))

100%|██████████| 4000/4000 [19:28<00:00,  3.42it/s]


In [13]:
Y = df['diagnosis']
Ya = np.array(Y)

In [14]:
print(X.shape)
print(Ya.shape)

(4000, 224, 224, 3)
(4000,)


In [15]:
from sklearn.model_selection import train_test_split

Xtr, Xtst, Ytr, Ytst = train_test_split(X,Ya, test_size=0.2, random_state=42, stratify=Ya)

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
from tensorflow.keras.utils import to_categorical

Yc = to_categorical(Ytr)

In [26]:
Ytst.shape

(800,)

In [27]:
# model.fit(Xtr, Yc, validation_split=0.2, batch_size=16, epochs=5)

model.fit(Xtr, Yc, epochs=5, batch_size=16, verbose=0,
          validation_split=0.2)
# model.fit(Xtr, Yc, epochs=5, batch_size=16, verbose=1, validation_data(X_val, y_val)

ValueError: You are passing a target array of shape (3200, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.

In [18]:
# import tensorflow as tf
# import keras_preprocessing
# from keras_preprocessing import image
# from keras_preprocessing.image import ImageDataGenerator

# TRAINING_DIR = "/tmp/rps/"
# training_datagen = ImageDataGenerator(
#     rescale = 1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest')

# VALIDATION_DIR = "/tmp/rps-test-set/"
# validation_datagen = ImageDataGenerator(rescale = 1./255)

# train_generator = training_datagen.flow_from_directory(
# TRAINING_DIR,
# target_size=(150,150),
# class_mode='categorical',
# batch_size=126
# )

# validation_generator = validation_datagen.flow_from_directory(
# VALIDATION_DIR,
# target_size=(150,150),
# class_mode='categorical',
# batch_size=126
# )

# model = tf.keras.models.Sequential([
#   # Note the input shape is the desired size of the image 150x150 with 3 bytes color
#   # This is the first convolution
#   tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
#   tf.keras.layers.MaxPooling2D(2, 2),
#   # The second convolution
#   tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#   tf.keras.layers.MaxPooling2D(2,2),
#   # The third convolution
#   tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#   tf.keras.layers.MaxPooling2D(2,2),
#   # The fourth convolution
#   tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#   tf.keras.layers.MaxPooling2D(2,2),
#   # Flatten the results to feed into a DNN
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dropout(0.5),
#   # 512 neuron hidden layer
#   tf.keras.layers.Dense(512, activation='relu'),
#   tf.keras.layers.Dense(3, activation='softmax')
# ])


# model.summary()

# model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)

# model.save("rps.h5")

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/rps/'